In [1]:
## Some Data

a = "purple is the best city in the forest"
b = "there is an art to getting your way and throwing bananas on to the street is not it"  # this is very similar to 'g'
c = "it is not often you find soggy bananas on the street"
d = "green should have smelled more tranquil but somehow it just tasted rotten"
e = "joyce enjoyed eating pancakes with ketchup"
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled"
g = "to get your way you must not bombard the road with yellow fruit"  # this is very similar to 'b'

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

/home/peng_luh/__git/search_l3s/search_l3s_search_srv/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# docs = [a, b, c, d, e, f, g]

import json

with open("/home/peng_luh/__git/search_l3s/search_l3s_search_srv/datasets/mls-tasks/json/data.json") as f:
    data = json.load(f)

docs = []
for i in range(len(data)):
    docs.append(data[i].get('contents'))
    
# docs

In [27]:
tokens = tokenizer("docs", max_length=512, truncation=True, padding='max_length', return_tensors='pt', add_special_tokens=True)

tokens.keys()
tokens.get("input_ids").shape

torch.Size([1, 512])

In [13]:
input_ids = tokens.get('input_ids')[0].unsqueeze(0)

In [14]:
outputs = model(input_ids)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [15]:
outputs.get('last_hidden_state').shape

torch.Size([1, 512, 768])

In [16]:
embeddings = outputs.last_hidden_state

# the embedding of sentence a
print(embeddings.shape)
embeddings

torch.Size([1, 512, 768])


tensor([[[-0.4055,  1.1942,  0.7918,  ..., -0.5296,  0.7376,  0.3884],
         [-0.4264,  0.9390,  0.8876,  ..., -0.2034,  0.5843,  0.2582],
         [-0.4308,  0.8892,  1.3110,  ..., -0.4963,  0.8360,  0.3146],
         ...,
         [ 0.1689,  1.0010,  0.4724,  ..., -0.1787,  0.1866, -0.1446],
         [ 0.1976,  0.9855,  0.5090,  ..., -0.2269,  0.2191, -0.1096],
         [ 0.0189,  1.2638,  0.6223,  ..., -0.3361,  0.2688, -0.0729]]],
       grad_fn=<NativeLayerNormBackward0>)

In [20]:
## mean pooling to create the sentece vector

# 1. multiply each value in embeddings tensor by its respective attention_mask value.

masks = tokens.get('attention_mask')[0].unsqueeze(-1).expand(embeddings.size()).float()
print(masks.shape)
masks

torch.Size([1, 512, 768])


tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [24]:
masked_embeddings = embeddings * masks
print(masked_embeddings.shape)
torch.sum(masked_embeddings, 1).shape

torch.Size([1, 512, 768])


torch.Size([1, 768])

In [22]:
masked_embeddings

tensor([[[-0.4055,  1.1942,  0.7918,  ..., -0.5296,  0.7376,  0.3884],
         [-0.4264,  0.9390,  0.8876,  ..., -0.2034,  0.5843,  0.2582],
         [-0.4308,  0.8892,  1.3110,  ..., -0.4963,  0.8360,  0.3146],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.0000,  0.0000, -0.0000]]],
       grad_fn=<MulBackward0>)

In [22]:
masked_embeddings = embeddings * masks
masked_embeddings.shape

torch.Size([7, 512, 768])

In [23]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([7, 768])

In [24]:
counted = torch.clamp(masks.sum(1), min=1e-9)
counted.shape

torch.Size([7, 768])

In [25]:
mean_pooled = summed / counted

mean_pooled[0]

tensor([-6.4737e-01, -2.2722e-01,  2.8184e-01,  4.9897e-01, -6.9554e-02,
        -8.5028e-01,  5.3480e-01,  4.9781e-01,  4.2247e-01, -8.1101e-01,
        -3.3428e-01, -8.4455e-01,  2.6676e-01,  3.6907e-01,  6.6989e-01,
         2.0275e-01, -3.1590e-01,  9.6454e-02,  8.3629e-01, -6.4574e-01,
        -1.7763e-01, -4.8748e-01,  1.8015e-02, -5.8890e-01,  8.6877e-01,
        -4.7323e-01,  1.6984e-01, -3.7928e-02,  5.0701e-01,  3.2727e-01,
         1.2181e-01,  5.0407e-01, -3.9131e-01, -5.4687e-01,  8.7298e-01,
         1.0905e+00, -8.5406e-01, -3.7172e-01,  2.3221e-01, -8.2706e-01,
         8.6060e-01, -6.6310e-01,  5.0642e-01, -7.8033e-01, -1.2961e+00,
        -8.5810e-02, -1.5043e+00,  1.1714e+00,  9.5849e-01, -7.5900e-01,
        -1.1085e-02,  7.5281e-02,  3.8400e-01,  3.4750e-01, -8.3249e-01,
         2.6063e-01, -6.0691e-02,  1.5899e-01,  1.2728e-01, -4.8628e-01,
        -7.4357e-01, -4.9982e-01, -3.9453e-01, -2.0274e-01, -2.4762e-01,
         5.8482e-01,  8.4303e-01,  5.1525e-01, -6.9

In [16]:
from sklearn.metrics.pariwise import cosine_similarity
import numpy as np

# convert to numpy array from torch tensor
mean_pooled = mean_pooled.detach().numpy()

scores = np.zeros(mean_pooled.shape[0], mean_pooled.shape[0])

for i in range(mean_pooled.shape[0]):
    scores[i, :] = cosine_similarity(mean_pooled[i], mean_pooled)[0]

scores

ModuleNotFoundError: No module named 'sklearn.metrics.pariwise'

In [30]:
A = torch.tensor([[[1, 1], 
                   [2, 2]]])

counted_A = torch.clamp(A.sum(1), min=1e-9)

counted_A

tensor([[3., 3.]])